In [ ]:
import openslide
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from openslide.deepzoom import DeepZoomGenerator
import os
import imageio
from PIL import Image
import datetime
import json
import cv2
from tqdm import tqdm

import skimage
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, hinge_loss
from tensorflow.keras.callbacks import CSVLogger
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
import csv
from sklearn import set_config
from sklearn.utils import check_random_state
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from joblib import dump, load
import joblib


import os
import shutil



In [ ]:
def create_dir(path):
    """ Create a directory. """
    if not os.path.exists(path):
        os.makedirs(path)

## Chemins

In [ ]:
main_directory = "C:/Users/Administrateur/ProjetM1/" # A MODIFIER : chemin principal vers /ProjetM1

# A MODIFIER MANUELLEMENT : chemin où trouver la slide à tester
WSI = "001A.tif" 
WSI_path = os.path.join(main_directory,"WSI_to_predict",WSI) #le chemin de la slide qui se trouve dans le dossier ProjetM1/WSI_to_predict


In [ ]:
name_WSI, ext_WSI = os.path.splitext(WSI)
patient = name_WSI[:3]

save_dir = os.path.join(main_directory, 'WSI_saved')
patches_dir = os.path.join(save_dir, patient, 'patches')


# Découper la slide en patch et sauvegarder dans /WSI_saved/patches

In [ ]:
#Creating data saved folder
create_dir(save_dir)

#Creating patches folder
create_dir(patches_dir)

In [ ]:
#___________alternative pour tester directement un patient plutôt qu'une seule slide___________

# A MODIFIER MANUELLEMENT : chemin où trouver le patient à tester
#patient = "001" 
#patient_path = os.path.join(main_directory,"patient_to_predict",patient) #le chemin des slides


#for slide in patient folder :
    #load WSI
    #path = os.path.join(patient_path, slide)
    #slide = openslide.open_slide(WSI_path)
    #...

In [ ]:
#load WSI
slide = openslide.open_slide(WSI_path)

# --------------------------------------------------------------------
# CREATION TISSUE MASK
# --------------------------------------------------------------------

# Dimensions de l'image
width, height = slide.dimensions
#print("The dimensions of the " + WSI + " slide at level 0 are :\n Width : " + str(width) + "\n Height : " + str(height))

# Niveaux de zoom disponibles
levels = slide.level_count

# Niveau de zoom souhaité
level = 4 #3

# Charger l'image à ce niveau
image = slide.read_region((0, 0), level, slide.level_dimensions[level])

# Convertir l'image en un tableau numpy
image = np.array(image)

# Convertir l'image en niveaux de gris
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Seuillage
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)  

mask = gray < thresh
mask_tissue = mask.astype(int)*255

# main directory to save patches
save_dir

#size of patches
patch_size = (512, 512)

# other parameters
threshold = 204
level0_dim = slide.level_dimensions[0]
#test = 0


# --------------------------------------------------------------------
# PATCHING THE WSI
# --------------------------------------------------------------------


# Read patch by patch according to coodinates (with WSI level 0 dimensions)
for y in tqdm(range(0, level0_dim[1], patch_size[1]), desc="COLONNES"):
    for x in range(0, level0_dim[0], patch_size[0]):
        #remove borders
        if x == 0 or x == 512 or x ==1024 or x==level0_dim[0] or x == level0_dim[0]-patch_size[0] or x == level0_dim[0]-2*patch_size[0] or y == 0 or y == 512 or y ==1024 or y==level0_dim[1] or y == level0_dim[1]-patch_size[1] or y == level0_dim[1]-2*patch_size[1] :
            continue

        # masking the image with tissue mask : go through mask (acccording to patch by patch coordinates)
        mask_patch = mask_tissue[int(y/16):int((y+patch_size[1])/16),int(x/16):int((x+patch_size[1])/16)]

        # calculate color average of the patch
        if mask_patch.any():
            average = mask_patch.mean()
        else:
            average = 0
        #print(WSI +" Average pixel value for TISUE mask patch (%d,%d) is: %.2f" % (x, y, average)) # if patch is a white area in mask_tissue ( = tissue)

        if average > threshold:
            # ______creation of patches_______
            # read corresponding region in WSI
            patch = slide.read_region((x, y), 1, patch_size)
            tile_name = os.path.join(patches_dir, '%d_%d' % (x, y))
            tile_name = tile_name.replace('\\','/')
            patch.save(tile_name + ".png")

# closing 
slide.close()


COLONNES: 100%|██████████████████████████████████████████████████████████████████████| 353/353 [22:27<00:00,  3.82s/it]


# SGD classifier cancéreux normal & garder les patch cancéreux pour la suite

In [ ]:
def CreateTuple1(patches_dir):
        
    patch_type_tuple = []

    for patch in os.listdir(patches_dir):
        patch_path = os.path.join(patches_dir,patch)
        patch_type_tuple.append((patient, patch_path))

                
    return patch_type_tuple

testfinal_tuple_1 =  CreateTuple1(patches_dir) 
print(testfinal_tuple_1)

[('001', 'C:/Users/Administrateur/ProjetM1/WSI_saved\\001\\patches\\10240_118784.png'), ('001', 'C:/Users/Administrateur/ProjetM1/WSI_saved\\001\\patches\\10240_119296.png'), ('001', 'C:/Users/Administrateur/ProjetM1/WSI_saved\\001\\patches\\10240_119808.png'), ('001', 'C:/Users/Administrateur/ProjetM1/WSI_saved\\001\\patches\\10240_120320.png'), ('001', 'C:/Users/Administrateur/ProjetM1/WSI_saved\\001\\patches\\10240_120832.png'), ('001', 'C:/Users/Administrateur/ProjetM1/WSI_saved\\001\\patches\\10240_121344.png'), ('001', 'C:/Users/Administrateur/ProjetM1/WSI_saved\\001\\patches\\10240_139776.png'), ('001', 'C:/Users/Administrateur/ProjetM1/WSI_saved\\001\\patches\\10240_140288.png'), ('001', 'C:/Users/Administrateur/ProjetM1/WSI_saved\\001\\patches\\10240_140800.png'), ('001', 'C:/Users/Administrateur/ProjetM1/WSI_saved\\001\\patches\\10240_141312.png'), ('001', 'C:/Users/Administrateur/ProjetM1/WSI_saved\\001\\patches\\10240_141824.png'), ('001', 'C:/Users/Administrateur/ProjetM1/

In [ ]:
print(len(testfinal_tuple_1))

9419


In [ ]:
def lpb(patch_type_tuple):

    # Créer des listes vides pour stocker les caractéristiques et les étiquettes
    features = []
    removed = []
    countelem = 0

    # Charger les images depuis les chemins spécifiés dans la liste patch_type_tuple
    
    for elem in tqdm(patch_type_tuple):
        path = elem[1]
        img_bgr = cv2.imread(path, 1)
        
        try:
            height, width, _ = img_bgr.shape
        except AttributeError:
            print(f"Error occurred while processing {path}. Skipping to the next iteration.")
            removed.append(countelem) # tableau contenant les index des éléments non pris en compte dans le tuple
            continue
            
        countelem += 1
        img = cv2.cvtColor(img_bgr,cv2.COLOR_BGR2GRAY)
        skimage.feature.local_binary_pattern(img, 8, 1, method='default')
        
        # Taille des patches
        patch_size = 32

        # Boucle pour extraire les patches et calculer les histogrammes
        histograms = []
        for y in range(0, img.shape[0], patch_size):
            for x in range(0, img.shape[1], patch_size):
                # Extraction du patch
                patch = img[y:y+patch_size, x:x+patch_size]


                # Création d'un masque pour le patch
                mask = np.zeros(patch.shape[:2], np.uint8)
                mask[:] = 255


                # Calcul de l'histogramme pour le patch
                hist = cv2.calcHist([patch], [0], mask, [256], [0, 256])


                # Normalisation de l'histogramme
                cv2.normalize(hist, hist)


                # Ajout de l'histogramme normalisé à la liste
                histograms.extend(hist)


        # Concaténation des histogrammes normalisés dans un vecteur
        histogram_vector = np.concatenate(histograms)
        
        # Aplatir chaque image en une ligne de caractéristiques
        features.append(histogram_vector)

     
    return features, removed

In [ ]:
clf_loaded_1 = joblib.load(os.path.join(main_directory,'model1b_SGD_trained_batch.joblib'))# prendre le SGD qui nous intéresse 

features_testfinal1, removed = lpb(testfinal_tuple_1)#nom a changer apres le découpage


C:\ProgramData\Anaconda3\envs\Projet18_env\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
 15%|███████████▍                                                                  | 1387/9419 [02:50<11:38, 11.50it/s]

Error occurred while processing C:/Users/Administrateur/ProjetM1/WSI_saved\001\patches\2560_100864.png. Skipping to the next iteration.


100%|██████████████████████████████████████████████████████████████████████████████| 9419/9419 [19:19<00:00,  8.12it/s]


In [ ]:
sum_patches = len(testfinal_tuple_1) - len(removed)

num_batches = 10
batch_size = int(sum_patches/num_batches)  #définir la taille du lot (batch size)


# predict by batch
listPred = []
for i in range(num_batches):
    batch_labels = clf_loaded_1.predict(features_testfinal1[int(i * batch_size):int((i + 1) * batch_size)])
    listPred.extend(batch_labels)

# dernier batch pour bien prendre toutes les données restantes
batch_labels = clf_loaded_1.predict(features_testfinal1[num_batches * batch_size:])
listPred.extend(batch_labels)
    


In [ ]:
print("Le nombre de patch utilisés = ", sum_patches)
print("Le nombre de patch au total = ", len(testfinal_tuple_1))
print("Index des patchs non utilisés car erronés au moment du LPB :", removed)
print(len(listPred))

Le nombre de patch utilisés =  9418
Le nombre de patch au total =  9419
Index des patchs non utilisés car erronés au moment du LPB : [1385]
9418


In [ ]:
import os 
import pandas as pd

testfinal_tuple_2=[]
count = 0
for i in range(len(testfinal_tuple_1)):
    
    if i == len(listPred): #on s'arrête quand on arrive à la taille de listPred
        break
    
    if i in removed: #on n'enregistre rien pour ne pas décaler les correspondances testfinal_tuple_1 et testfinal_tuple_2
        continue
    
    if listPred[i]=="cancerous":
        testfinal_tuple_2.append((patient, testfinal_tuple_1[i][1]))
    

print("Nombre de patch cancéreux = ", len(testfinal_tuple_2))

Nombre de patch cancéreux =  491


## SGD pour classifier chaque patch entre ABC et GCB 

In [ ]:
clf_loaded_2 = joblib.load(os.path.join(main_directory,'SGD_trained_batch.joblib'))# prendre le SGD qui nous intéresse 
features_testfinal2, removed2 = lpb(testfinal_tuple_2)#nom a changer apres le découpage


sum_patches = len(testfinal_tuple_2) - len(removed2)

num_batches = 10
batch_size = int(sum_patches/num_batches)  #définir la taille du lot (batch size)

# predict by batch
listPred2 = []

for i in range(num_batches):
    batch_labels = clf_loaded_2.predict(features_testfinal2[i * batch_size:(i + 1) * batch_size])
    listPred2.extend(batch_labels)
# dernier batch pour bien prendre toutes les données restantes
batch_labels = clf_loaded_2.predict(features_testfinal2[num_batches * batch_size:])
listPred.extend(batch_labels)

C:\ProgramData\Anaconda3\envs\Projet18_env\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
100%|████████████████████████████████████████████████████████████████████████████████| 491/491 [00:54<00:00,  8.96it/s]


### Comparer le nombre de ABC et de GCB et délivrer un resultat final

**REGLE DE PREDICTION DU RESULTAT FINAL**

Si un patient possède un nombre de patches ABC inférieur à 20% (***seuil à définir***) de son nombre de patches GCB alors le cancer est de type ABC (et vice versa).

In [ ]:
count_ABC = 0
count_GCB = 0
for i in range(len(listPred2)):
    if listPred2[i]=="ABC":
        count_ABC += 1
    else :
        count_GCB += 1

print("Nombre de prédictions ABC :", count_ABC)
print("Nombre de prédictions GCB :",count_GCB)

treshold = 0.2

if count_ABC < count_GCB * treshold : 
    print("Le cancer est de type GCB")
elif count_GCB < count_ABC * treshold :
    print("Le cancer est de type GCB")
else :
    print("Le type de cancer n'a pas pu être déterminé.")
    

Nombre de prédictions ABC : 476
Nombre de prédictions GCB : 14
le cancer est de type ABC
